In [15]:
import os
import pandas as pd
import warnings
from causalnex.structure.notears import from_pandas
from causalnex.structure.dynotears import from_pandas_dynamic

warnings.filterwarnings("ignore")  # silence warnings

In [11]:
inputs_path = os.path.join(os.path.dirname(os.getcwd()), "data", "inputs")
data_name = "etfs_macro_large"

data = pd.read_csv(os.path.join(inputs_path, f'{data_name}.csv'))
data['date'] = pd.to_datetime(data['date']) 
data.set_index("date", inplace=True)

In [14]:
sm = from_pandas(data)

In [17]:
dsm = from_pandas_dynamic(data, p=1)

TypeError: Index must be integers

In [18]:
data

,SPY,XLI,XLE,XLK,XLV,XLU,XLF,XLY,XLP,XLB,...,CUSR0000SAC,CUSR0000SAD,CUSR0000SAS,CPIULFSL,CUSR0000SA0L2,CUSR0000SA0L5,PCEPI,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA
date,,,,,,,,,,,,,,,,,,,,,
2000-01-31,0.026777,0.006920,0.023149,0.032666,-0.004659,0.045092,0.027474,0.007494,0.018325,-0.006013,...,0.001364,-0.001595,0.004175,0.003536,0.001844,0.003042,0.002725,-0.000403,0.002196,0.003556
2000-02-29,0.009596,0.011007,0.022964,0.022625,0.011167,0.007105,0.006853,0.020896,-0.003804,-0.006678,...,0.006116,-0.001597,0.002601,0.004109,0.004902,0.003639,0.003131,-0.002778,0.007243,0.002975
2000-03-31,0.011285,0.001604,-0.005318,0.018751,0.017992,0.005864,0.006461,-0.010621,-0.005138,-0.000677,...,-0.004032,-0.000797,0.002069,-0.000582,-0.001215,-0.001203,-0.000836,-0.001512,-0.004051,0.000457
2000-04-30,-0.006226,-0.006832,-0.029853,0.002844,0.009320,-0.000548,0.020619,-0.025836,-0.028945,0.007745,...,-0.004032,-0.000797,0.002069,-0.000582,-0.001215,-0.001203,-0.000836,-0.001512,-0.004051,0.000457
2000-05-31,0.002191,0.006909,0.029486,-0.019466,0.010650,0.005514,0.011925,0.023325,-0.009745,-0.008652,...,0.000000,0.003185,0.002581,0.001164,0.001215,0.001805,0.000836,0.000103,-0.001753,0.001923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,-0.007638,-0.008465,-0.009151,-0.009785,-0.006392,-0.006570,-0.007536,-0.009918,-0.006136,-0.012388,...,-0.004755,0.003005,0.006746,0.001421,0.000255,0.002013,0.002679,0.004809,-0.008099,0.005841
2022-09-30,-0.015584,-0.013191,-0.008021,-0.019344,-0.013858,-0.019500,-0.009180,-0.019878,-0.017529,-0.002643,...,0.005658,-0.006425,0.004330,0.004571,0.003748,0.005603,0.004115,-0.003914,0.008315,0.004255
2022-10-31,-0.007249,-0.002858,0.008368,-0.013284,-0.000828,-0.009390,-0.006161,-0.006436,-0.005894,-0.009004,...,0.005658,-0.006425,0.004330,0.004571,0.003748,0.005603,0.004115,-0.003914,0.008315,0.004255
